In [1]:
# Importing EdgeSimPy components
from edge_sim_py import *

# Importing Python libraries
import os
import random
import msgpack
import pandas as pd

In [2]:
# Functions
## my algorithm
def my_algorithm(parameters):
    # We can always call the 'all()' method to get a list with all created instances of a given class
    for service in Service.all():
        # We don't want to migrate services are are already being migrated
        if service.server == None and not service.being_provisioned:

            # Let's iterate over the list of edge servers to find a suitable host for our service
            for edge_server in EdgeServer.all():

                # We must check if the edge server has enough resources to host the service
                if edge_server.has_capacity_to_host(service=service):

                    # Start provisioning the service in the edge server
                    service.provision(target_server=edge_server)

                    # After start migrating the service we can move on to the next service
                    break

## stop criteria
def stopping_criterion(model: object):
    # Defining a variable that will help us to count the number of services successfully provisioned within the infrastructure
    provisioned_services = 0
    
    # Iterating over the list of services to count the number of services provisioned within the infrastructure
    for service in Service.all():

        # Initially, services are not hosted by any server (i.e., their "server" attribute is None).
        # Once that value changes, we know that it has been successfully provisioned inside an edge server.
        if service.server != None:
            provisioned_services += 1
    
    # As EdgeSimPy will halt the simulation whenever this function returns True, its output will be a boolean expression
    # that checks if the number of provisioned services equals to the number of services spawned in our simulation
    return provisioned_services == Service.count()

# Monitoring Custom Metrics
def custom_collect_method(self) -> dict:
    temperature = random.randint(10, 50)  # Generating a random integer between 10 and 50 representing the switch's temperature
    metrics = {
        "Instance ID": self.id,
        "Power Consumption": self.get_power_consumption(),
        "Temperature": temperature,
    }
    return metrics

# Overriding the NetworkSwitch's collect() method
NetworkSwitch.collect = custom_collect_method

In [3]:
# Creating a Simulator object
simulator = Simulator(
    dump_interval=5,
    tick_duration=1,
    tick_unit="seconds",
    stopping_criterion=stopping_criterion,
    resource_management_algorithm=my_algorithm,
)

# Loading a sample dataset from GitHub
simulator.initialize(input_file="https://raw.githubusercontent.com/EdgeSimPy/edgesimpy-tutorials/master/datasets/sample_dataset2.json")

# Executing the simulation
simulator.run_model()

In [4]:
# Option 1 (Accessing Variables Directly)
print(simulator.agent_metrics["User"][0:2])

[{'Object': 'User_1', 'Time Step': 0, 'Instance ID': 1, 'Coordinates': [6, 0], 'Base Station': 'BaseStation_4 ([6, 0])', 'Delays': {'1': None}, 'Communication Paths': {}, 'Making Requests': {'1': {'1': True}}, 'Access History': {'1': [{'start': 1, 'end': inf, 'duration': inf, 'waiting_time': 0, 'access_time': 0, 'interval': 0, 'next_access': inf}]}}, {'Object': 'User_2', 'Time Step': 0, 'Instance ID': 2, 'Coordinates': [3, 1], 'Base Station': 'BaseStation_6 ([3, 1])', 'Delays': {'2': None}, 'Communication Paths': {}, 'Making Requests': {'2': {'1': True}}, 'Access History': {'2': [{'start': 1, 'end': inf, 'duration': inf, 'waiting_time': 0, 'access_time': 0, 'interval': 0, 'next_access': inf}]}}]


In [5]:
# Option 2 (Accessing Log Files)

# Gathering the list of msgpack files in the current directory
logs_directory = f"{os.getcwd()}/logs"
dataset_files = [file for file in os.listdir(logs_directory) if ".msgpack" in file]

# Reading msgpack files found
datasets = {}
for file in dataset_files:
    with open(f"logs/{file}", "rb") as data_file:
        datasets[file.replace(".msgpack", "")] = pd.DataFrame(msgpack.unpackb(data_file.read(), strict_map_key=False))

# Let's access the edge server logs:
print(datasets["EdgeServer"].head())

         Object  Time Step  Instance ID Coordinates  Available  CPU    RAM  \
0  EdgeServer_1          0            1      [0, 0]       True    8  16384   
1  EdgeServer_2          0            2      [0, 2]       True    8  16384   
2  EdgeServer_3          0            3      [6, 0]       True    8   8192   
3  EdgeServer_4          0            4      [1, 3]       True    8   8192   
4  EdgeServer_5          0            5      [7, 1]       True   12  16384   

     Disk  CPU Demand  RAM Demand  Disk Demand  Ongoing Migrations Services  \
0  131072           0           0            0                   0       []   
1  131072           0           0            0                   0       []   
2  131072           0           0            0                   0       []   
3  131072           0           0            0                   0       []   
4  131072           1        1024         1017                   0       []   

  Registries                                            

In [6]:
# Defining the data frame columns that will be exhibited
properties = ['Coordinates', 'CPU Demand', 'RAM Demand', 'Disk Demand', 'Services']
columns = ['Time Step', 'Instance ID'] + properties

dataframe = datasets["EdgeServer"].filter(items=columns)
print(dataframe.head())

   Time Step  Instance ID Coordinates  CPU Demand  RAM Demand  Disk Demand  \
0          0            1      [0, 0]           0           0            0   
1          0            2      [0, 2]           0           0            0   
2          0            3      [6, 0]           0           0            0   
3          0            4      [1, 3]           0           0            0   
4          0            5      [7, 1]           1        1024         1017   

  Services  
0       []  
1       []  
2       []  
3       []  
4       []  


In [7]:
# Creating a Pandas data frame with the network switch logs
logs = pd.DataFrame(simulator.agent_metrics["NetworkSwitch"])
print(logs.head())

            Object  Time Step  Instance ID  Power Consumption  Temperature
0  NetworkSwitch_1          0            1               60.6           41
1  NetworkSwitch_2          0            2               61.2           20
2  NetworkSwitch_3          0            3               61.2           44
3  NetworkSwitch_4          0            4               60.9           32
4  NetworkSwitch_5          0            5               61.5           24
